In [3]:
%matplotlib tk
%reload_ext autoreload
import cv2 as cv
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from src.preprocessing import *
import shutil
import sys
from collections import defaultdict

# Dataset generation

In [89]:
def generate_pacthes(image_dir : list, concentration : list, frame : list):

    
    images_dir = {mM : path for mM, path in zip(sorted(concentration),sorted(image_dir))}
    
    
    images_path = {
        mM: sorted([img for img in os.listdir(images_dir[mM]) if img.endswith('.tif')])
        for mM in sorted(concentration)
    }
    first_image = {mM : images_path[mM][0] for mM in sorted(concentration)}
    print(first_image)
    images_path = {mM : sorted([images_path[mM][min(int(idx * len(images_path[mM])), len(images_path[mM]) - 1)] for idx in sorted(frame)]) for mM in sorted(concentration)}
    images = {mM : [cv.imread(os.path.join(images_dir[mM],img), cv.IMREAD_GRAYSCALE) for img in images_path[mM]] for mM in sorted(concentration)}
    delta = {mM : [cv.absdiff(img,cv.imread(os.path.join(images_dir[mM],first_image[mM]),cv.IMREAD_GRAYSCALE)) for img in images[mM]] for mM in sorted(concentration)}

    # liste à n images avec n correspondant au nombre de frame indiqué (0.2,0.5,0.75,1) correspondent à 4 frames positionner au 1/4, 1/2, 3/4 , fin
    # ici idx correspond au pourcentage par rapport à la quantité d'image totale 1 -> dernière frame, 0.5 -> frame du millieu ...
    
    patches= []
    patches_delta = []
    for mM in sorted(concentration):
        for img, img_delta, frm in zip(tqdm(images[mM]), delta[mM],frame):
            preprocessed_img = img.astype(np.float32) / 255.0
            preprocessed_img_delta = img_delta.astype(np.float32) / 255.0

            
            patches.append(extract_patches(preprocessed_img,concentration=mM, frame=frm))
            patches_delta.append(extract_patches(preprocessed_img_delta,concentration=mM, frame=frm))

    merged = {}
    merged_delta = {}
    for dic, dic_delta in zip(patches,patches_delta):
        merged.update(dic)
        merged_delta.update(dic_delta)

    return merged, merged_delta

# Dossier avec les patches delta

In [90]:
patches, patches_delta = generate_pacthes(['/home/ters-user/Documents/images_data/30mM_2/','/home/ters-user/Documents/images_data/50mM/'],['30mM(2)','50mM'],[0.25,0.5,0.75,0.95]) #List of dictionnaries : each dictionnary contains the patches of the considered frame

{'30mM(2)': '30mM_1.6V-w-snaps-18-27-56.446.tif', '50mM': '50mM_1.6V-w-snaps-18-55-11.257.tif'}


100%|██████████| 4/4 [00:57<00:00, 14.38s/it]


In [91]:
out = '/home/ters-user/Documents/particle_recognition_using_CNN/dataset/'

keys_delta, images_delta = list(patches_delta.keys()), list(patches_delta.values())
keys, images = list(patches.keys()), list(patches.values())

index = 0

count = 0

while True:
    filepath = os.path.join(out,f'{keys[index]}.png')
    img_delta = images_delta[index][0]
    img = images[index][0]

    if img_delta is None:
        print(f"Erreur de lecture : {keys[index]}")
        continue

    display = img_delta.copy()
    cv.putText(display, f"{count}" , (10, 30),
                cv.FONT_HERSHEY_SIMPLEX, 0.8, 255, 2)

    cv.imshow("display",display)
    key = cv.waitKey(0) & 0xFF

    if key == ord('q'):
        break
    elif key == 81 or key == ord('j'):  
        index = max(0, index - 1)
    elif key == 83 or key == ord('l'):  
        index = min(len(images) - 1, index + 1)
    elif key == ord('a'):
        cv.imwrite(filepath, img*255)
        cv.imwrite(f"{filepath.split('.p')[0]}_delta.png", img_delta*255)
        count += 1

cv.destroyAllWindows()

#Fichier avec les images (delta, original) choisies (les plus intéréssantes)

In [92]:
from segments import SegmentsClient, SegmentsDataset
from segments.utils import export_dataset

In [93]:
client = SegmentsClient('cd10b0a22912bf3f2d6c9f215d687876a914ad75')
release = client.get_release('BamBoombap/Corrosion', 'v1.0') 
dataset = SegmentsDataset(release, labelset='ground-truth', filter_by=['labeled', 'reviewed'])

# Export to semantic format (each pixel has)
export_dataset(dataset, export_format='semantic')

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 88/88 [00:00<00:00, 164.86it/s]


Initialized dataset with 88 images.
Exporting dataset. This may take a while...


  0%|          | 0/88 [00:00<?, ?it/s]/home/ters-user/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/skimage/util/dtype.py:576: UserWarning: Downcasting int64 to uint8 without scaling because max value 1 fits in uint8
  return _convert(image, np.uint8, force_copy)
100%|██████████| 88/88 [00:00<00:00, 924.48it/s]

Exported to segments/BamBoombap_Corrosion/v1.0


'segments/BamBoombap_Corrosion/v1.0'

In [22]:
masks_path = '/home/ters-user/Documents/particle_recognition_using_CNN/dataset/'

masks = os.listdir(masks_path)

In [18]:
for mask in masks:
    name = os.path.join(masks_path,mask)
    os.rename(name, f'{name.split('_delta')[0]}_mask.png')

In [23]:
for mask in masks:
    name = os.path.join(masks_path,mask)
    img = cv.imread(name,cv.IMREAD_UNCHANGED)
    img = img*255.0
    cv.imwrite(name,img)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [

In [24]:
raw_dir = '/home/ters-user/Documents/particle_recognition_using_CNN/brouillon/'

raw = os.listdir(raw_dir)

for mask in masks:
    if f"{mask.split('_mask')[0]}.png" in raw:
        
        source = os.path.join(raw_dir, f"{mask.split('_mask')[0]}.png")
        destination = os.path.join(masks_path, f"{mask.split('_mask')[0]}.png")
        
        shutil.copy(source, destination)
        
        

In [29]:
data_dir = '/home/ters-user/Documents/particle_recognition_using_CNN/dataset/'
reference = '/home/ters-user/Documents/particle_recognition_using_CNN/brouillon/'

data = os.listdir(data_dir)
ref = os.listdir(reference)

for img in ref:
    if 'delta' not in img:
        if img not in data:
            print(img)
            
            mask = np.zeros((200,200),dtype=np.uint8)

            source = os.path.join(reference,img)
            destination = os.path.join(data_dir,img)
            shutil.copy(source, destination)

            cv.imwrite(f"{destination.split('.')[0]}_mask.png",mask)
    

30mM(2)_0.25_53.000.png
30mM(2)_0.25_102.000.png
30mM(2)_0.25_436.000.png
30mM(2)_0.25_143.000.png
30mM(2)_0.5_268.000.png
30mM(2)_0.25_140.000.png
30mM(2)_0.5_272.000.png
30mM(2)_0.25_142.000.png
30mM(2)_0.25_1.000.png
30mM(2)_0.25_0.000.png
30mM(2)_0.25_141.000.png
30mM(2)_0.5_250.000.png


In [43]:
mask = np.zeros((200,200), dtype=np.uint8)
cv.imwrite(os.path.join(data_dir,'30mM(2)_0.5_250.000.png'),mask)

True